In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import date
import questionary

In [2]:
stocks = ['AAPL', 'AMZN', 'FB', 'TSLA', '^GSPC']

start_date = '2010-01-01'

end_date = '2017-01-01'

data = {}

for stock in stocks:
    ticker = yf.download(stock, start_date, end_date)
    # Adjusted Closing Price takes into account factors such as dividends, stock splits, etc.
    # Adjusted Closing Price is a more accurate reflection of the stock's value
    data[stock] = ticker['Adj Close']

pd.DataFrame(data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,AAPL,AMZN,FB,TSLA,^GSPC
Date,,,,,
2010-01-04,6.562591,133.899994,NaN,NaN,1132.989990
2010-01-05,6.573935,134.690002,NaN,NaN,1136.520020
2010-01-06,6.469369,132.250000,NaN,NaN,1137.140015
2010-01-07,6.457407,130.000000,NaN,NaN,1141.689941
2010-01-08,6.500339,133.520004,NaN,NaN,1144.979980
...,...,...,...,...,...
2016-12-23,27.459557,760.590027,117.269997,42.667999,2263.790039
2016-12-27,27.633949,771.400024,118.010002,43.905998,2268.879883
2016-12-28,27.516117,772.130005,116.919998,43.948002,2249.919922


In [3]:
stocks_data = pd.DataFrame(data)

stocks_data = stocks_data.fillna(stocks_data.rolling(6, min_periods=1).mean())
stocks_data = stocks_data.dropna()

stocks_data = stocks_data.resample('M').last()
stocks_data

,AAPL,AMZN,FB,TSLA,^GSPC
Date,,,,,
2012-05-31,17.716019,212.910004,29.600000,5.900000,1310.329956
2012-06-30,17.908285,228.350006,31.100000,6.258000,1362.160034
2012-07-31,18.728878,233.300003,21.709999,5.484000,1379.319946
2012-08-31,20.487083,248.270004,18.059999,5.704000,1406.579956
2012-09-30,20.544365,254.320007,21.660000,5.856000,1440.670044
2012-10-31,18.333790,232.889999,21.110001,5.626000,1412.160034
2012-11-30,18.106920,252.050003,28.000000,6.764000,1416.180054
2012-12-31,16.463844,250.869995,26.620001,6.774000,1426.189941
2013-01-31,14.091583,265.500000,30.980000,7.502000,1498.109985


In [4]:
sp500 = stocks_data.iloc[:,-1]
sp500 = pd.DataFrame(sp500)

In [5]:
sp500.head()

,^GSPC
Date,
2012-05-31,1310.329956
2012-06-30,1362.160034
2012-07-31,1379.319946
2012-08-31,1406.579956
2012-09-30,1440.670044


In [6]:
sp500['market_adjclose'] = sp500.sum(axis=1)
sp500.head()

,^GSPC,market_adjclose
Date,,
2012-05-31,1310.329956,1310.329956
2012-06-30,1362.160034,1362.160034
2012-07-31,1379.319946,1379.319946
2012-08-31,1406.579956,1406.579956
2012-09-30,1440.670044,1440.670044


In [7]:
stocks = stocks_data.iloc[:,:-1]
stocks = pd.DataFrame(stocks)
stocks.head()

,AAPL,AMZN,FB,TSLA
Date,,,,
2012-05-31,17.716019,212.910004,29.600000,5.900
2012-06-30,17.908285,228.350006,31.100000,6.258
2012-07-31,18.728878,233.300003,21.709999,5.484
2012-08-31,20.487083,248.270004,18.059999,5.704
2012-09-30,20.544365,254.320007,21.660000,5.856


In [8]:
weights = [0.25, 0.3, 0.1, 0.4]
print(weights)

[0.25, 0.3, 0.1, 0.4]


In [9]:
stocks_norm_weight = np.log(stocks) 
stocks_norm_weight.head()

,AAPL,AMZN,FB,TSLA
Date,,,,
2012-05-31,2.874469,5.360870,3.387774,1.774952
2012-06-30,2.885263,5.430880,3.437208,1.833861
2012-07-31,2.930067,5.452325,3.077773,1.701835
2012-08-31,3.019795,5.514517,2.893700,1.741168
2012-09-30,3.022587,5.538593,3.075467,1.767467


In [ ]:
weighted_stocks = stocks.mul(weights)
weighted_stocks.head()

In [ ]:
# we have to have a sector return value to compare against the market return
# so we sum up all the stocks in the sector per day to figure out the daily_return compared to the market
stocks['sector_adjclose'] = stocks.sum(axis=1)
stocks.head()

In [ ]:
data = pd.DataFrame({'stock_adjclose':stocks['sector_adjclose'], 'market_adjclose':sp500['market_adjclose']})
data.head()

In [ ]:
    def initialize(self):
            stocks_data = self.download_data()
            # we use monthly returns ('M') instead of daily returns
            stocks_data = stocks_data.resample('M').last()

            # create a pandas dataframe to store stock information for analysis
            self.data = pd.DataFrame({'stock_adjclose':stocks_data[self.stocks[0]], 'market_adjclose':stocks_data[self.stocks[-1]]})

            # add 2 columns for the s_returns and m_returns
            # logarithmic monthly returns
            self.data[['stock_returns', 'market_returns']] = np.log(self.data[['stock_adjclose', 'market_adjclose']] / self.data[['stock_adjclose', 'market_adjclose']].shift(1))

            # remove NaN values
            self.data =  self.data[1:]

            print(self.data)